# Agenți AI Azure cu suport pentru Protocolul de Context al Modelului (MCP)

Acest notebook demonstrează cum să utilizați agenții AI Azure cu instrumente Protocolul de Context al Modelului (MCP) pentru a crea un agent inteligent care poate valorifica servere MCP externe pentru capabilități îmbunătățite.


## Instalarea pachetelor necesare NuGet

Mai întâi, trebuie să instalăm pachetul Azure AI Agents Persistent, care oferă funcționalitatea de bază pentru lucrul cu Azure AI Agents.


## Beneficiile Autentificării fără Cheie

Acest notebook demonstrează **autentificarea fără cheie**, care oferă mai multe avantaje:
- ✅ **Fără chei API de gestionat** - Utilizează autentificarea bazată pe identitatea Azure
- ✅ **Securitate îmbunătățită** - Fără secrete stocate în cod sau configurație
- ✅ **Rotirea automată a acreditivelor** - Azure gestionează ciclul de viață al acreditivelor
- ✅ **Acces bazat pe roluri** - Utilizează Azure RBAC pentru permisiuni detaliate

`DefaultAzureCredential` va utiliza automat cea mai bună sursă de acreditive disponibilă:
1. Identitate gestionată (când rulează în Azure)
2. Acreditivele Azure CLI (în timpul dezvoltării)
3. Acreditivele Visual Studio
4. Variabilele de mediu (dacă sunt configurate)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Instalați pachetul Azure Identity pentru autentificare cu serviciile Azure folosind DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importați Spațiile de Nume Necesare

Importați spațiile de nume necesare pentru Azure AI Agents și Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Configurarea Azure AI Agent Client (Autentificare fără cheie)

Configurați variabilele necesare și creați PersistentAgentsClient utilizând **autentificarea fără cheie**:
- **projectEndpoint**: Endpoint-ul proiectului Azure AI Foundry
- **modelDeploymentName**: Numele modelului AI implementat (GPT-4.1 nano)
- **mcpServerUrl**: URL-ul serverului MCP (Microsoft Learn API)
- **mcpServerLabel**: O etichetă pentru identificarea serverului MCP
- **DefaultAzureCredential**: Utilizează identitatea gestionată, Azure CLI sau alte surse de acreditive (nu sunt necesare chei API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Creare Definiție Instrument MCP

Creează o definiție de instrument MCP care se conectează la serverul Microsoft Learn MCP. Acest lucru va permite agentului să acceseze conținutul și documentația Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Crearea agentului AI

Creează un agent AI persistent cu modelul și instrumentele MCP specificate. Agentul este configurat cu:
- Modelul GPT-4.1 nano
- Instrucțiuni pentru utilizarea instrumentelor MCP pentru asistență
- Acces la serverul Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Creează un fir de conversație și trimite un mesaj

Creează un fir de conversație și trimite un mesaj utilizatorului, întrebând despre diferența dintre Azure OpenAI și OpenAI. Acest lucru va testa abilitatea agentului de a utiliza instrumentele MCP pentru a oferi informații precise.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Configurarea Resurselor Instrumentului MCP (Fără Cheie)

Configurați resursele instrumentului MCP. Pentru o abordare cu adevărat fără cheie, puteți elimina anteturile personalizate dacă serverul MCP acceptă autentificarea bazată pe identitatea Azure. Exemplul de mai jos arată cum să adăugați anteturi dacă este necesar, dar pentru scenarii fără cheie, acestea pot să nu fie necesare.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Pornire Agent Run

Creează și pornește o execuție pentru a procesa mesajul utilizatorului. Agentul va folosi instrumentele și resursele MCP configurate pentru a genera un răspuns.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Monitorizează Execuția și Gestionează Aprobările pentru Instrumente (Fără Cheie)

Monitorizează starea execuției agentului și gestionează orice aprobare necesară pentru instrumente. Acest ciclu:
1. Așteaptă finalizarea execuției sau necesitatea unei acțiuni
2. Aprobă automat apelurile instrumentului MCP atunci când este necesar
3. Pentru autentificarea fără cheie, anteturile pot să nu fie necesare dacă serverul MCP suportă identitatea Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Afișează Rezultatele Conversației

Recuperează și afișează toate mesajele din firul conversației, arătând atât întrebarea utilizatorului, cât și răspunsul agentului. Mesajele sunt afișate în ordine cronologică, cu marcaje de timp și indicatoare de rol.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
